## In this file is described the loop that is done to determine dN/dy, <pT> for every centrality of the Table

#### The whole file is highly inspired by the Exp_fit_lowpT file. This is just the same programm but pu in a funciton that loop over the items of the ALICE data. I suppressed the explanation comments on this programm.

In [1]:
import ROOT as r
import numpy as np
import math 
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
PbPb_pp_file = r.TFile("ROOTfile/PbPb_pp5tev.root")
PbPb_pp_file

In [3]:
Histo = PbPb_pp_file.Get("Table 5")
# Histo.ls()

In [4]:
def exp_fct(x, par):
    
    pt = x[0]
    dNdy = par[0]
    T = par[1]
    mp = par[2]

    mt = math.sqrt(pt*pt+mp*mp)

    Factor = dNdy * pt * (1/(T*(mp + T))) 

    Expression = math.exp(-(mt-mp)/T)

    return Factor * Expression


In [5]:
def dNdy_compute(TF1, err_stat, err_syst): 

    dndy_estimate = 0
    low_uncertainties_stat = 0
    up_uncertainties_stat = 0
    low_uncertainties_syst = 0
    up_uncertainties_syst = 0

    bin_area = 0
    low_error_syst = 0
    up_error_syst = 0
    low_error_stat = 0
    up_error_stat = 0

    Jmeas =  0
    sigmaJ_meas_square_stat = 0
    sigmaJ_meas_square_syst = 0
    pt = 0
    

    for i in range(1, TF1.GetNbinsX()+1):

        pt = TF1.GetXaxis().GetBinCenter(i)
        bin_area = TF1.GetBinContent(i) * TF1.GetBinWidth(i)  


        low_error_stat = err_stat.GetBinContent(i) * TF1.GetBinWidth(i) 
        up_error_stat = err_stat.GetBinContent(i) * TF1.GetBinWidth(i) 

        low_error_syst = err_syst.GetBinContent(i) * TF1.GetBinWidth(i) 
        up_error_syst = err_syst.GetBinContent(i) * TF1.GetBinWidth(i)

        low_uncertainties_stat = low_uncertainties_stat + low_error_stat**2    #sum the lowests uncertainties
        up_uncertainties_stat = up_uncertainties_stat + up_error_stat**2    #sum the upper uncertainties

        low_uncertainties_syst = low_uncertainties_syst + low_error_syst**2   
        up_uncertainties_syst = up_uncertainties_syst + up_error_syst**2 

        sigmaJ_meas_square_stat = sigmaJ_meas_square_stat + (pt * up_error_stat)**2
        sigmaJ_meas_square_syst = sigmaJ_meas_square_syst + (pt * up_error_syst)**2

        
        dndy_estimate = dndy_estimate + bin_area  #This part sum the area of each bin --> dN/dy measured
        Jmeas = Jmeas + pt * bin_area


    return dndy_estimate, math.sqrt(up_uncertainties_stat), math.sqrt(low_uncertainties_stat), math.sqrt(up_uncertainties_syst), math.sqrt(low_uncertainties_syst), Jmeas, math.sqrt(sigmaJ_meas_square_stat), math.sqrt(sigmaJ_meas_square_syst)

In [6]:
def adapt_error(TF1, err1, err2, err3):

    for i in range(1, TF1.GetNbinsX()+1): #nb of bins
        err = math.sqrt( err1.GetBinContent(i)**2 +  err3.GetBinContent(i)**2 )  ##We didn't take now into account correlated uncertainties now
        TF1.SetBinError(i, err) 

In [7]:
def adapt_error_shift(TF1, err1, err2, err3):

    for i in range(1, TF1.GetNbinsX()+1): #nb of bins
        
        err = math.sqrt( err1.GetBinContent(i)**2 +  err3.GetBinContent(i)**2 )  ##We didn't take now into account correlated uncertainties now
        TF1.SetBinError(i, err) 
        value = TF1.GetBinContent(i)
        TF1.SetBinContent(i, value + err2.GetBinContent(i) )

In [8]:
def compute_one_centrality_y(Histo, iy, cent_label, exp_fct, dNdy_compute, adapt_error, adapt_error_shift):

    ytag = f"y{iy}"

 
    TGraphAsymm = Histo.Get(f"Graph1D_{ytag};1")
    Y1 = Histo.Get(f"Hist1D_{ytag};1")

    e1 = Histo.Get(f"Hist1D_{ytag}_e1")
    e2 = Histo.Get(f"Hist1D_{ytag}_e2")
    e3 = Histo.Get(f"Hist1D_{ytag}_e3")

    n = TGraphAsymm.GetN()

    for i in range(n):
        err = math.sqrt(e1.GetBinContent(i+1)**2 + e3.GetBinContent(i+1)**2)
        exl = TGraphAsymm.GetErrorXlow(i)
        exh = TGraphAsymm.GetErrorXhigh(i)
        TGraphAsymm.SetPointError(i, exl, exh, err, err)

    fit_min = 0.3
    fit_max = 2.5

    m_p = 0.938272

    exp = r.TF1(f"exp1_{iy}", exp_fct, fit_min, fit_max, 3)
    exp.SetParNames("dNdy", "T", "mp")
    exp.SetParameters(10, 0.35, m_p)
    exp.FixParameter(2, m_p)
    exp.SetParLimits(1, 1e-4, 5.0) 


    fit_exp = TGraphAsymm.Fit(exp,"RS0")

    exp.SetRange(0.0, fit_max)
    I_extrapolated =exp.Integral(0.0, fit_min)

    g   = r.TF1(f"g_{iy}", lambda x, p: x[0]*exp.EvalPar(x, p), 0, fit_max, exp.GetNpar())

    for i in range(exp.GetNpar()):
        g.SetParameter(i, exp.GetParameter(i))

    J_ext = g.Integral(0.0, fit_min)

    adapt_error(Y1, e1, e2, e3)
    dndy_mes = dNdy_compute(Y1, e1, e3)

    I_meas = dndy_mes[0]
    sigma_stat_meas = dndy_mes[1]
    sigma_syst_meas = dndy_mes[3]

    J_meas = dndy_mes[5]
    sigmaJ_meas_stat = dndy_mes[6]
    sigmaJ_meas_syst = dndy_mes[7]

    Itot = I_meas + I_extrapolated
    Jtot = J_meas + J_ext
    pt_mean = Jtot/Itot

    sigmaI_ext = exp.IntegralError( 0.0, fit_min, fit_exp.GetParams(), fit_exp.GetCovarianceMatrix().GetMatrixArray())

    sigmaJ_ext = g.IntegralError( 0.0, fit_min, fit_exp.GetParams(), fit_exp.GetCovarianceMatrix().GetMatrixArray())


    TGraphAsymm_shift = Histo.Get(f"Graph1D_{ytag};1")
    Y1_shift = Histo.Get(f"Hist1D_{ytag};1")

    n = TGraphAsymm_shift.GetN()

    for i in range(n):

        err = math.sqrt(e1.GetBinContent(i+1)**2 + e3.GetBinContent(i+1)**2)

        exl = TGraphAsymm_shift.GetErrorXlow(i)
        exh = TGraphAsymm_shift.GetErrorXhigh(i)

        TGraphAsymm_shift.SetPointError(i, exl, exh, err, err)

        point = TGraphAsymm_shift.GetPointY(i)
        
        TGraphAsymm_shift.SetPointY(i, point + e2.GetBinContent(i+1))

    adapt_error_shift(Y1_shift, e1, e2, e3)
    
    exp_shift = r.TF1(f"exp1_shift_{iy}", exp_fct, fit_min, fit_max, 3)
    exp_shift.SetParNames("dNdy", "T", "mp")
    exp_shift.SetParameters(10, 0.35, 0.938272)
    exp_shift.FixParameter(2, 0.938272)
    exp_shift.SetParLimits(1, 1e-4, 5.0) 


    fit_exp_shift = TGraphAsymm_shift.Fit(exp_shift,"RS0")
    
    exp_shift.SetRange(0.0, fit_max)
    I_err_extra_extrapolated = exp_shift.Integral(0.0, fit_min)

    g_shift   = r.TF1(f"g_shift_{iy}", lambda x, p: x[0]*exp_shift.EvalPar(x, p), 0, fit_max, exp_shift.GetNpar())

    for i in range(exp_shift.GetNpar()):
        g_shift.SetParameter(i, exp_shift.GetParameter(i))

    J_ext_shift = g_shift.Integral(0.0, fit_min)

    dNdy_shift = dNdy_compute(Y1_shift, e1, e2)

    I_meas_shift = dNdy_shift[0]
    J_meas_shift = dNdy_shift[5]

    I_tot_shift = I_meas_shift + I_err_extra_extrapolated
    J_tot_shift = J_meas_shift + J_ext_shift

    pt_shift = J_tot_shift/I_tot_shift

    Ishift_tot = I_meas_shift + I_err_extra_extrapolated

    sigma_corr = Ishift_tot-Itot

    sigma_syst = math.sqrt(sigma_corr**2 + dndy_mes[3]**2)
    sigma_stat = math.sqrt(sigmaI_ext**2 + dndy_mes[1]**2)

    sigmaI_stat_pt = math.sqrt(sigmaI_ext**2 + sigma_stat_meas**2) 
    sigmaJ_stat_pt = math.sqrt(sigmaJ_ext**2 + sigmaJ_meas_stat**2) 

    sigma_stat_pt = math.sqrt( (sigmaJ_stat_pt/Itot)**2 + ((Jtot * sigmaI_stat_pt)/((Itot)**2))**2 )
    sigma_syst_uncorr_pt = math.sqrt( (sigmaJ_meas_syst/Itot)**2 + ((Jtot * sigma_syst_meas)/((Itot)**2))**2 )

    sigma_syst_corr_pt = np.abs(pt_mean-pt_shift)
    sigma_syst_pt = math.sqrt( sigma_syst_uncorr_pt**2 + sigma_syst_corr_pt**2)

    return {
        "centrality": cent_label,
        "dNdy": Itot,
        "dNdy_stat": sigma_stat,
        "dNdy_syst": sigma_syst,
        "<pT>": pt_mean,
        "pT_stat": sigma_stat_pt,
        "pT_syst": sigma_syst_pt,
    }
  




In [10]:
centrality_labels = [ "0-5%", "5-10%", "10-20%", "20-30%", "30-40%", "40-50%", "50-60%", "60-70%", "70-80%", "80-90%"]

results = []

for i in range(1,11):   # y1 → y10

    out = compute_one_centrality_y(Histo,i, centrality_labels[i-1], exp_fct, dNdy_compute, adapt_error, adapt_error_shift)
    results.append(out)
 
df = pd.DataFrame(results)
df

,centrality,dNdy,dNdy_stat,dNdy_syst,<pT>,pT_stat,pT_syst
0,0-5%,75.308378,0.073061,3.678460,1.435737,0.002005,0.014900
1,5-10%,62.039479,0.076979,2.902608,1.422816,0.002625,0.012118
2,10-20%,47.786128,0.048390,2.190451,1.404363,0.002093,0.011299
3,20-30%,33.389036,0.039971,1.495349,1.368982,0.002419,0.010798
4,30-40%,22.622337,0.033077,1.006451,1.315466,0.002810,0.009993
5,40-50%,14.527163,0.026427,0.656278,1.251926,0.003295,0.010354
6,50-60%,8.728685,0.020632,0.396503,1.180458,0.003996,0.010011
7,60-70%,4.734391,0.015838,0.264583,1.101634,0.005150,0.012193
8,70-80%,2.295198,0.011723,0.128253,1.020755,0.007078,0.011441
9,80-90%,0.905967,0.006893,0.050435,0.954920,0.010027,0.011506


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      54.4797
NDf                       =           25
Edm                       =  2.15035e-09
NCalls                    =           78
dNdy                      =      86.8453   +/-   1.31541     
T                         =     0.763486   +/-   0.0119952    	 (limited)
mp                        =     0.938272                      	 (fixed)
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      69.3067
NDf                       =           25
Edm                       =  1.31308e-07
NCalls                    =           78
dNdy                      =      89.7345   +/-   1.3322      
T                         =     0.747024   +/-   0.0116326    	 (limited)
mp                        =     0.938272                      	 (fixed)
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      75.0789
ND